## Prepare everything and fit the AutoGluon predictor

In [1]:
!pip install -U autogluon > /dev/null

from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np

df_train = pd.read_csv('/kaggle/input/playground-series-s4e1/train.csv')
df_orig = pd.read_csv('/kaggle/input/bank-customer-churn-prediction/Churn_Modelling.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e1/test.csv')

# Include the original dataset & do some processing
cols = ['CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender', 'Age', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember','EstimatedSalary', 'Exited']
df_train = pd.concat((df_train, df_orig), axis=0).drop(['id', 'RowNumber'], axis=1).drop_duplicates().dropna()
df_train = df_train.loc[df_train.groupby(cols)['Tenure'].idxmax()].drop_duplicates()

# Add the length of Surname string
df_train['Surname_len'] = df_train['Surname'].apply(len)
df_test['Surname_len'] = df_test['Surname'].apply(len)

train = TabularDataset(df_train)
test = TabularDataset(df_test)

automl = TabularPredictor(label='Exited', problem_type='binary', eval_metric='roc_auc')
automl.fit(train, presets='best_quality')

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.8.0 requires botocore<1.33.2,>=1.32.4, but you have botocore 1.29.165 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.1 which is incompatible.
beatrix-jupyterlab 2023.814.150030 requires jupyter-server~=1.16, but you have jupyter-server 2.12.1 which is incompatible.
beatrix-jupyterlab 2023.814.150030 requires jupyterlab~=3.4, but you have jupyterlab 4.0.5 which is incompatible.
conda 23.11.0 requires packaging>=23.0, but you have packaging 21.3 which is incompatible.
gcsfs 2023.6.0 requires fsspec==2023.6.0, but you have fsspec 2023.12.2 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have

No path specified. Models will be saved in: "AutogluonModels/ag-20240112_195201"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240112_195201/ds_sub_fit/sub_fit_ho.
2024-01-12 19:52:02,050	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Beginning Au

## Explore the leaderboard

In [2]:
automl.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.896180,roc_auc,99.907760,2284.928053,0.030982,57.095116,3,True,21
1,CatBoost_BAG_L2,0.895915,roc_auc,77.359704,1898.492632,0.823546,232.348445,2,True,19
2,WeightedEnsemble_L2,0.894660,roc_auc,45.073611,1494.313935,0.030111,38.884467,2,True,14
3,LightGBMXT_BAG_L2,0.894513,roc_auc,84.251141,1792.423436,7.714982,126.279248,2,True,15
4,CatBoost_BAG_L1,0.894112,roc_auc,0.866602,780.300793,0.866602,780.300793,1,True,7
5,ExtraTreesGini_BAG_L2,0.894005,roc_auc,83.274487,1691.799827,6.738329,25.655639,2,True,20
6,LightGBMXT_BAG_L1,0.892319,roc_auc,31.496265,188.345777,31.496265,188.345777,1,True,3
7,RandomForestEntr_BAG_L2,0.891947,roc_auc,84.244703,1824.249275,7.708545,158.105087,2,True,18
8,LightGBM_BAG_L2,0.891909,roc_auc,79.538105,1734.539207,3.001947,68.395020,2,True,16
9,RandomForestGini_BAG_L2,0.890754,roc_auc,84.606359,1811.723767,8.070201,145.579579,2,True,17


## Make a 'pure' submission file

In [3]:
prediction = automl.predict_proba(test)

data_submit = pd.read_csv('/kaggle/input/playground-series-s4e1/sample_submission.csv')
data_submit.Exited = prediction[1]
data_submit[['id', 'Exited']].to_csv('simple_ag.csv', index=False)

!head simple_ag.csv

INFO:sklearnex: sklearn.neighbors.KNeighborsClassifier.predict_proba: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.neighbors.KNeighborsClassifier.predict_proba: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


id,Exited
165034,0.010571056976914406
165035,0.7929214239120483
165036,0.0161212757229805
165037,0.19829848408699036
165038,0.36204731464385986
165039,0.05699170008301735
165040,0.015554123558104038
165041,0.07332485914230347
165042,0.6069941520690918


## Add external solutions to produce an ensemble
- https://www.kaggle.com/code/arunklenin/ps4e1-advanced-feature-engineering-ensemble 
- https://www.kaggle.com/code/ravi20076/playgrounds4e01-baseline-v2

In [4]:
ext1 = pd.read_csv('/kaggle/input/ps4e1-advanced-feature-engineering-ensemble/submission_pure.csv')
ext2 = pd.read_csv('/kaggle/input/playgrounds4e01-baseline-v2/Submission_V1.csv')

# Beware of overfitting ;)
data_submit.Exited = data_submit.Exited + ext1.Exited + ext2.Exited
data_submit[['id', 'Exited']].to_csv('ensemble.csv', index=False)

!head ensemble.csv

id,Exited
165034,0.049367425272764304
165035,2.29528553070136
165036,0.0690874780346635
165037,0.5819774118536575
165038,1.108805130655356
165039,0.21366357522301535
165040,0.08178077457187964
165041,0.23673312966517895
165042,1.8216824173970196
